In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# Menggunakan tanda seru (!) untuk menjalankan perintah shell
!git clone https://github.com/GreenAvo-Capstone/Machine-Learning.git

Cloning into 'Machine-Learning'...
remote: Enumerating objects: 15054, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 15054 (delta 16), reused 307 (delta 13), pack-reused 14739
Receiving objects: 100% (15054/15054), 412.14 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (29425/29425), done.


In [ ]:
import os

# Cek apakah direktori dataset sudah ada
dataset_dir = './Machine-Learning/dataset'
if os.path.exists(dataset_dir):
    print("Dataset berhasil diunduh.")
else:
    print("Dataset tidak ditemukan.")


Dataset berhasil diunduh.


In [ ]:
path_train = os.path.join(dataset_dir, 'train')
path_val = os.path.join(dataset_dir, 'val')
path_test = os.path.join(dataset_dir, 'test')

In [ ]:
BATCH_SIZE = 32
img_size = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(path_train, shuffle=True, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")
valid_ds = tf.keras.utils.image_dataset_from_directory(path_val, shuffle=True, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")
test_ds = tf.keras.utils.image_dataset_from_directory(path_test, shuffle=False, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")

Found 11032 files belonging to 5 classes.
Found 2205 files belonging to 5 classes.
Found 1473 files belonging to 5 classes.


In [ ]:
# Clear previous sessions
tf.keras.backend.clear_session()

# Define image size
img_size = 224

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_train,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_val,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_test,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

# Get class names from training dataset
class_names = train_ds.class_names
num_class = len(class_names)

Found 11032 files belonging to 5 classes.
Found 2205 files belonging to 5 classes.
Found 1473 files belonging to 5 classes.


In [ ]:
# Load MobileNetV3Large with pre-trained weights and make it trainable
base_model = tf.keras.applications.MobileNetV3Large(input_shape=(img_size, img_size, 3), include_top=False, pooling="max")
base_model.trainable = True

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((img_size, img_size, 3), name="Input", dtype=tf.float32),
    tf.keras.layers.Rescaling(scale=1./255, name="Rescale_Layer"),
    base_model,
    tf.keras.layers.Dense(128, activation='selu', kernel_initializer="lecun_normal", name="FC"),
    tf.keras.layers.Dense(num_class, activation='softmax')
])

# Compile the model
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='precision'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='recall')
    ]
)

# Apply additional preprocessing if needed
def preprocess(image, label):
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

    train_ds = train_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)
    valid_ds = valid_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)

# Check the shapes of a batch of images and labels
for images, labels in train_ds.take(1):
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

# Define callbacks
    bestCB = tf.keras.callbacks.ModelCheckpoint(filepath=f'best_model.h5', verbose=1, save_best_only=True)
    lrCB = tf.keras.callbacks.ReduceLROnPlateau(verbose=1, min_lr=1e-20, patience=5)
    esCB = tf.keras.callbacks.EarlyStopping(patience=45, verbose=1)

12683000/12683000 [==============================] - 1s 0us/step
Images shape: (32, 224, 224, 3)
Labels shape: (32,)


In [ ]:
history = model.fit(
    train_ds,
    validation_data = valid_ds,
    epochs = 100,
    callbacks=[bestCB, lrCB, esCB]
)

Epoch 1/100
345/345 [==============================] - ETA: 0s - loss: 1.1761 - accuracy: 0.6171 - precision: 0.6171 - recall: 0.6171
Epoch 1: val_loss improved from inf to 3.56506, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


345/345 [==============================] - 92s 162ms/step - loss: 1.1761 - accuracy: 0.6171 - precision: 0.6171 - recall: 0.6171 - val_loss: 3.5651 - val_accuracy: 0.1946 - val_precision: 0.1946 - val_recall: 0.1946 - lr: 1.0000e-04
Epoch 2/100
345/345 [==============================] - ETA: 0s - loss: 0.6070 - accuracy: 0.7517 - precision: 0.7517 - recall: 0.7517
Epoch 2: val_loss did not improve from 3.56506
345/345 [==============================] - 51s 147ms/step - loss: 0.6070 - accuracy: 0.7517 - precision: 0.7517 - recall: 0.7517 - val_loss: 4.7817 - val_accuracy: 0.1946 - val_precision: 0.1946 - val_recall: 0.1946 - lr: 1.0000e-04
Epoch 3/100
345/345 [==============================] - ETA: 0s - loss: 0.4487 - accuracy: 0.8192 - precision: 0.8192 - recall: 0.8192
Epoch 3: val_loss improved from 3.56506 to 2.75477, saving model to best_model.h5
345/345 [==============================] - 56s 161ms/step - loss: 0.4487 - accuracy: 0.8192 - precision: 0.8192 - recall: 0.8192 - val_lo

KeyboardInterrupt: 

In [ ]:
model.save("./checkpoint/latest/")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label = "Training Loss")
plt.plot(history.history['val_loss'], label = "Validation Loss")
plt.legend(loc="best")
plt.title("Loss History")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.plot(history.history['precision'], label = "Training Precision")
plt.plot(history.history['val_precision'], label = "Validation Precision")
plt.legend(loc="best")
plt.title("Precision History")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = history.epoch

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model's Accuracy")
plt.legend()
plt.show()

plt.plot(epochs[10:], loss[10:], label='Train Loss')
plt.plot(epochs[10:], val_loss[10:], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model's Loss")
plt.legend()
plt.show()

In [ ]:
# Load iterasi terakhir
# model = tf.keras.models.load_model("./checkpoint/latest/")

# Load terbaik
best_model = tf.keras.models.load_model(f'best_model.h5')

# Evaluasi model pada data testing
loss, accuracy, precision, recall = best_model.evaluate(test_ds)
print("Loss pada data testing:", loss)
print("Akurasi pada data testing:", accuracy)
print("Precision pada data testing:", precision)
print("Recall pada data testing:", recall)

In [ ]:
# Prediksi kelas untuk setiap gambar di data testing
predictions = best_model.predict(test_ds)
predicted_classes = tf.math.argmax(predictions, axis=1)

In [ ]:
# Tampilkan beberapa contoh prediksi
for images, labels in test_ds.take(1):
    for i in range(5):  # Tampilkan 5 gambar pertama
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Prediksi: {class_names[predicted_classes[i]]}, \
                  Aktual: {class_names[labels[i]]}")
        plt.show()

In [ ]:
# Simpan Model
model.save('model_alpukat.h5')

# Convert the Model to TensorFlow Lite

In [ ]:
# Optimize size
converter = tf.lite.TFLiteConverter.from_saved_model(f'best_model.h5')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

tflite_file = pathlib.Path('model size.tflite')
tflite_file.write_bytes(tflite_model)

In [ ]:
# Optimize latency
converter = tf.lite.TFLiteConverter.from_saved_model(f'best_model.h5')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
tflite_model = converter.convert()

tflite_file = pathlib.Path('model latency.tflite')
tflite_file.write_bytes(tflite_model)